# Linear Evaluations


## Import Libraries

In [ ]:
COLAB = True
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # navigate to the folder where this notebook is located
    %%cd drive/MyDrive/Duke/DNN/final

In [ ]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
import pickle
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

# import shutil, time, os, requests, random, copy

import matplotlib.pyplot as plt
import seaborn as sns

# move one directory up to import from src
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")

# from src.setdevice import set_device
from src.dataloader_eval import DataLoader, GenericDataset
import warnings
warnings.filterwarnings("ignore")


## Set Seeds

In [5]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
set_seed()

In [6]:
def set_device():
    if torch.cuda.is_available():  # check if NVIDIA GPU is available
        device = torch.device("cuda")
        print(f"Running on {device} GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():  # check if Apple's Metal is available
        device = torch.device("mps")
        print(
            f"Checking pytorch is built with mps activated: {torch.backends.mps.is_built()}"
        )
        print(f"Running on {device} GPU...")
    else:
        device = torch.device("cpu")
        print("Running on CPU...")

    return device


device = set_device()


Running on cuda GPU: A100-SXM4-40GB


## Define f(·) encoder - ResNet20


In [8]:
"""
It is the neural network model base encoder f(·) that extracts representation vectors 
from augmented data examples. We choose to use the ResNet20 architecture.
"""

# Residual Block
class BasicBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=inplanes,
            out_channels=planes,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, out_channels=planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        if inplanes > planes or stride > 1:
            # Option B
            self.downsample = nn.Sequential(
                nn.Conv2d(
                    in_channels=inplanes,
                    out_channels=planes,
                    kernel_size=1,
                    stride=stride,
                    padding=0,
                    bias=False,
                ),
                nn.BatchNorm2d(planes),
            )
        else:
            self.downsample = None
        self.stride = stride

    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity if self.downsample is None else self.downsample(identity)
        out = F.relu(out)
        return out


# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv0 = nn.Conv2d(
            in_channels=3,
            out_channels=self.inplanes,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False,
        )
        self.bn0 = nn.BatchNorm2d(self.inplanes)
        self.layer1 = self._make_layer(block, planes=16, blocks=layers[0], stride=1)
        self.layer2 = self._make_layer(block, planes=32, blocks=layers[1], stride=2)
        self.layer3 = self._make_layer(block, planes=64, blocks=layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_layer(self, block, planes, blocks, stride):
        layers = []
        for i in range(blocks):
            layers.append(
                block(
                    inplanes=(self.inplanes if i == 0 else planes),
                    planes=planes,
                    stride=stride if i == 0 else 1,
                )
            )
        self.inplanes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn0(self.conv0(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        return x


## Define LinearEval

Only the encoder part of the model is used to evaluate

In [43]:
class LinearEval(nn.Module):
    def __init__(self, num_class, pretrained_path):
        super(LinearEval, self).__init__()

        # encoder
        self.encoder = ResNet(BasicBlock, [3, 3, 3]) #f(·) encoder
        # classifier
        self.fc = nn.Linear(64, 10, bias=True)
        self.load_state_dict(
            torch.load(pretrained_path, map_location="cpu"), strict=False
        )

    def forward(self, x):
        x = self.encoder(x)
        out = self.fc(x)
        return out


## Set up Hyperparameters for Eval

Range reference from paper:

LRS = [0.5, 1, 1.5]

BATCHES = [256, 512, 1024, 2048, 4096]

In [44]:
# Tunable hyperparameters
BATCH = 1024
LR = 0.001 * BATCH / 256
EPOCHS = 90
DECAY = 1e-4


## Load Best Model to Evaluate

In [27]:
# load best model
model_path = "results/B1024_E100_lars_with_schedulers_best_model.pth"

model = LinearEval(num_class=10, pretrained_path=model_path).to(device)

for param in model.encoder.parameters():
    param.requires_grad = False

optimizer = optim.Adam(model.fc.parameters(), lr=LR, weight_decay=DECAY) 
criterion = nn.CrossEntropyLoss() 

## Load Data
This loader only loads a percentage of data to test performance on limited data. This is the same as what is used in RotNet implementation

In [32]:
# Load limited data
LOAD_PCT = .10

train_set = GenericDataset(
    dataset_name='cifar10',
    split= 'train', # crop and horizontal flip 
    num_imgs_per_cat=int(LOAD_PCT*5000))
test_set = GenericDataset(
    dataset_name='cifar10',
    split='test') 

train_loader = DataLoader(
    dataset=train_set,
    batch_size=BATCH,
    unsupervised=False,
    epoch_size=10*5000,
    num_workers=4,
    shuffle=True)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=BATCH,
    unsupervised=False,
    epoch_size=None,
    num_workers=4,
    shuffle=False)

############################################

Files already downloaded and verified
num_imgs_per_category 500
Files already downloaded and verified


## Training Loop

In [ ]:
save_name_eval = "B1024_E100_lars_with_schedulers"

In [40]:
print(f"evaluating {int(100*LOAD_PCT)}pct data")
results = {
    "train_loss": [],
    "train_acc@1": [],
    "train_acc@5": [],
    "test_loss": [],
    "test_acc@1": [],
    "test_acc@5": [],
}

best_acc = 0.0

for epoch in range(1, EPOCHS + 1):

    print("")

    model.train()

    total_loss, total_correct_1, total_correct_5, total_num, data_bar = (
        0.0,
        0.0,
        0.0,
        0,
        tqdm(train_loader(0)),
    )
    with torch.enable_grad():
        for data, target in data_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)

            out = model(data)
            loss = criterion(out, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)

            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum(
                (prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()
            ).item()
            total_correct_5 += torch.sum(
                (prediction[:, 0:5] == target.unsqueeze(dim=-1)).any(dim=-1).float()
            ).item()

            data_bar.set_description(
                "{} Epoch: [{}/{}] Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%".format(
                    "Train",
                    epoch,
                    EPOCHS,
                    total_loss / total_num,
                    total_correct_1 / total_num * 100,
                    total_correct_5 / total_num * 100,
                )
            )

    train_loss = total_loss / total_num
    train_acc_1 = total_correct_1 / total_num * 100
    train_acc_5 = total_correct_5 / total_num * 100

    results["train_loss"].append(train_loss)
    results["train_acc@1"].append(train_acc_1)
    results["train_acc@5"].append(train_acc_5)

    model.eval()

    total_loss, total_correct_1, total_correct_5, total_num, data_bar = (
        0.0,
        0.0,
        0.0,
        0,
        tqdm(test_loader(0)),
    )
    with torch.no_grad():
        for data, target in data_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = model(data)
            loss = criterion(out, target)

            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)

            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum(
                (prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()
            ).item()
            total_correct_5 += torch.sum(
                (prediction[:, 0:5] == target.unsqueeze(dim=-1)).any(dim=-1).float()
            ).item()

            data_bar.set_description(
                "{} Epoch: [{}/{}] Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%".format(
                    "Test",
                    epoch,
                    EPOCHS,
                    total_loss / total_num,
                    total_correct_1 / total_num * 100,
                    total_correct_5 / total_num * 100,
                )
            )

    test_loss = total_loss / total_num
    test_acc_1 = total_correct_1 / total_num * 100
    test_acc_5 = total_correct_5 / total_num * 100

    results["test_loss"].append(test_loss)
    results["test_acc@1"].append(test_acc_1)
    results["test_acc@5"].append(test_acc_5)

    # # save statistics
    data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    data_frame.to_csv(f"./results/{save_name_eval}_lineval{int(100*LOAD_PCT)}pct_stats.csv", index_label='epoch')

    if test_acc_1 > best_acc:
        best_acc = test_acc_1
        # torch.save(model.state_dict(), '/workspaces/RepLearning/saved_models/linear_model.pth')
        torch.save(model.state_dict(), f"./results/{save_name_eval}_lineval{int(100*LOAD_PCT)}pct.pth")


evaluating 10 data



Train Epoch: [1/90] Loss: 1.0916 ACC@1: 62.96% ACC@5: 95.50%: 100%|██████████| 391/391 [00:06<00:00, 63.41it/s]
Test Epoch: [1/90] Loss: 0.9243 ACC@1: 67.27% ACC@5: 97.43%: 100%|██████████| 79/79 [00:01<00:00, 76.66it/s]


Train Epoch: [2/90] Loss: 1.0267 ACC@1: 64.35% ACC@5: 96.07%: 100%|██████████| 391/391 [00:06<00:00, 63.62it/s]
Test Epoch: [2/90] Loss: 0.9208 ACC@1: 67.11% ACC@5: 97.46%: 100%|██████████| 79/79 [00:01<00:00, 74.23it/s]


Train Epoch: [3/90] Loss: 1.0070 ACC@1: 64.68% ACC@5: 96.25%: 100%|██████████| 391/391 [00:06<00:00, 63.89it/s]
Test Epoch: [3/90] Loss: 0.9007 ACC@1: 67.63% ACC@5: 97.74%: 100%|██████████| 79/79 [00:01<00:00, 71.69it/s]


Train Epoch: [4/90] Loss: 0.9968 ACC@1: 65.33% ACC@5: 96.24%: 100%|██████████| 391/391 [00:06<00:00, 62.32it/s]
Test Epoch: [4/90] Loss: 0.8793 ACC@1: 68.34% ACC@5: 97.62%: 100%|██████████| 79/79 [00:01<00:00, 74.13it/s]


...

Train Epoch: [86/90] Loss: 0.9610 ACC@1: 66.25% ACC@5: 96.69%: 100%|██████████| 391/391 [00:06<00:00, 62.13it/s]
Test Epoch: [86/90] Loss: 0.8634 ACC@1: 68.85% ACC@5: 97.82%: 100%|██████████| 79/79 [00:01<00:00, 73.02it/s]


Train Epoch: [87/90] Loss: 0.9630 ACC@1: 66.25% ACC@5: 96.60%: 100%|██████████| 391/391 [00:06<00:00, 61.96it/s]
Test Epoch: [87/90] Loss: 0.8606 ACC@1: 68.99% ACC@5: 97.85%: 100%|██████████| 79/79 [00:01<00:00, 74.27it/s]


Train Epoch: [88/90] Loss: 0.9653 ACC@1: 66.17% ACC@5: 96.54%: 100%|██████████| 391/391 [00:06<00:00, 61.75it/s]
Test Epoch: [88/90] Loss: 0.8575 ACC@1: 68.94% ACC@5: 97.90%: 100%|██████████| 79/79 [00:01<00:00, 74.27it/s]


Train Epoch: [89/90] Loss: 0.9631 ACC@1: 66.19% ACC@5: 96.69%: 100%|██████████| 391/391 [00:06<00:00, 61.60it/s]
Test Epoch: [89/90] Loss: 0.8687 ACC@1: 68.45% ACC@5: 97.80%: 100%|██████████| 79/79 [00:01<00:00, 71.50it/s]


Train Epoch: [90/90] Loss: 0.9653 ACC@1: 66.09% ACC@5: 96.53%: 100%|██████████| 391/391 [00:06<00:00, 61.77it/s]
Test Epoch: [90/90] Loss: 0.8538 ACC@1: 69.30% ACC@5: 97.99%: 100%|██████████| 79/79 [00:01<00:00, 72.05it/s]


In [45]:
data_frame.tail()

,train_loss,train_acc@1,train_acc@5,test_loss,test_acc@1,test_acc@5
86,0.960983,66.252,96.690,0.863361,68.85,97.82
87,0.963018,66.254,96.604,0.860634,68.99,97.85
88,0.965272,66.172,96.538,0.857543,68.94,97.90
89,0.963108,66.188,96.688,0.868718,68.45,97.80
90,0.965289,66.094,96.530,0.853815,69.30,97.99
